In [1]:
import os
import glob
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import KFold, GroupKFold, cross_val_score, train_test_split
import optuna

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [3]:
numFolds = 5
wiFiIDFeatureDir = "referencePublicNotebooks/waypt_WiFiID1000Feat/"
wiFiDtFeatureDir = "referencePublicNotebooks/waypt_WiFiDt1000Feat/"

In [4]:
# the metric used in this competition
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

# get our train and test files
wiFiIDFeatureTrainFiles = sorted(glob.glob(os.path.join(wiFiIDFeatureDir, 'train/*_train.csv')))
wiFiDtFeatureTrainFiles = sorted(glob.glob(os.path.join(wiFiDtFeatureDir, 'train/*_train.csv')))

In [5]:
len(wiFiIDFeatureTrainFiles), len(wiFiDtFeatureTrainFiles)

(24, 24)

In [6]:
e = 0
wiFiIDdf = pd.read_csv(wiFiIDFeatureTrainFiles[e])
wiFiDtdf = pd.read_csv(wiFiDtFeatureTrainFiles[e])

In [7]:
#wiFiIDdf.head(3)
#wiFiDtdf.head(3)

In [8]:
wiFiIDFeature = wiFiIDdf.iloc[:,:-4].to_numpy()
wiFiDtFeature = wiFiDtdf.to_numpy()

# normalize to 1
wiFiDtFeature = wiFiDtFeature.astype(float) / 1000.0
wiFiIDFeature = wiFiIDFeature.astype(float) / 999.0
x_train = np.column_stack((wiFiIDFeature,wiFiDtFeature))
print(x_train.shape)

y_trainy = wiFiIDdf.iloc[:,-3].values
y_trainx = wiFiIDdf.iloc[:,-4].values
y_trainf = wiFiIDdf.iloc[:,-2].values
print(wiFiDtFeature.shape, wiFiIDFeature.shape, y_trainx.shape, y_trainy.shape, y_trainf.shape)

(1974, 1882)
(1974, 941) (1974, 941) (1974,) (1974,) (1974,)


In [9]:
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).

def objective(trial):        
    train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_trainy, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)
    param = {
        "objective": "regression",
        "metric": "l2",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    accuracy = sklearn.metrics.mean_squared_error(valid_y, preds)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    
    ## increase n_trails to test more params
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


[I 2021-03-28 15:10:00,945] A new study created in memory with name: no-name-36f3e848-bc5c-4a38-bf17-7fef200dd2c1
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:10:01,963] Trial 0 finished with value: 126.08038258224093 and parameters: {'lambda_l1': 5.942824626224296e-05, 'lambda_l2': 7.827627651264878e-05, 'n_estimators': 113, 'num_leaves': 38, 'feature_fraction': 0.6564770009785859, 'bagging_fraction': 0.43952028769020546, 'bagging_freq': 2, 'min_child_samples': 92}. Best is trial 0 with value: 126.08038258224093.
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 202

/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:10:59,779] Trial 13 finished with value: 40.22343677800098 and parameters: {'lambda_l1': 0.10406236562537848, 'lambda_l2': 0.15306812352881335, 'n_estimators': 139, 'num_leaves': 59, 'feature_fraction': 0.8986822417509279, 'bagging_fraction': 0.889994526173345, 'bagging_freq': 5, 'min_child_samples': 20}. Best is trial 3 with value: 34.676505385696004.
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:11:08,063] Trial 14 finished with value: 42.395807821452465 and parameters: {'lambda_l1': 0.00914086102007727

/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:12:13,289] Trial 26 finished with value: 43.39543209238769 and parameters: {'lambda_l1': 1.3596614179411656e-06, 'lambda_l2': 0.0006575447423491814, 'n_estimators': 126, 'num_leaves': 225, 'feature_fraction': 0.40802662768413694, 'bagging_fraction': 0.5358931659149582, 'bagging_freq': 7, 'min_child_samples': 10}. Best is trial 23 with value: 33.034451582100836.
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:12:28,719] Trial 27 finished with value: 48.20553083137827 and parameters: {'lambda_l1': 4.470987389

/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:13:33,621] Trial 39 finished with value: 125.22947395786073 and parameters: {'lambda_l1': 1.4422839025922627, 'lambda_l2': 1.1183594990915942e-06, 'n_estimators': 132, 'num_leaves': 161, 'feature_fraction': 0.7716364485539783, 'bagging_fraction': 0.46072940173811644, 'bagging_freq': 3, 'min_child_samples': 99}. Best is trial 23 with value: 33.034451582100836.
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:13:38,679] Trial 40 finished with value: 45.63035429561669 and parameters: {'lambda_l1': 1.97733586110

/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:15:32,112] Trial 52 finished with value: 53.79403087419138 and parameters: {'lambda_l1': 0.2977341539820253, 'lambda_l2': 1.650303994833352e-08, 'n_estimators': 171, 'num_leaves': 158, 'feature_fraction': 0.6451780803916141, 'bagging_fraction': 0.77928333401577, 'bagging_freq': 4, 'min_child_samples': 14}. Best is trial 23 with value: 33.034451582100836.
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:15:36,632] Trial 53 finished with value: 36.273307626617836 and parameters: {'lambda_l1': 9.225001704152328

/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:16:27,948] Trial 65 finished with value: 31.162408715775484 and parameters: {'lambda_l1': 9.95282546248911, 'lambda_l2': 0.8986085629363542, 'n_estimators': 159, 'num_leaves': 219, 'feature_fraction': 0.4991243487416595, 'bagging_fraction': 0.9722361152706888, 'bagging_freq': 4, 'min_child_samples': 22}. Best is trial 64 with value: 27.878290494803295.
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:16:31,443] Trial 66 finished with value: 43.616442431774516 and parameters: {'lambda_l1': 5.412405636460718, 

/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:17:17,644] Trial 78 finished with value: 41.538618986485986 and parameters: {'lambda_l1': 8.982882453091175, 'lambda_l2': 0.6589483304724981, 'n_estimators': 121, 'num_leaves': 230, 'feature_fraction': 0.43251784880700267, 'bagging_fraction': 0.9623798011143436, 'bagging_freq': 3, 'min_child_samples': 23}. Best is trial 64 with value: 27.878290494803295.
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:17:21,446] Trial 79 finished with value: 39.199445360013854 and parameters: {'lambda_l1': 0.866442346329931

/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:18:59,143] Trial 91 finished with value: 36.451981044982865 and parameters: {'lambda_l1': 0.5087731031959953, 'lambda_l2': 0.029498360978572054, 'n_estimators': 175, 'num_leaves': 207, 'feature_fraction': 0.4570593257676514, 'bagging_fraction': 0.9663929648263105, 'bagging_freq': 3, 'min_child_samples': 13}. Best is trial 64 with value: 27.878290494803295.
/home/wabco/miniconda3/envs/kaggle/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2021-03-28 15:19:05,818] Trial 92 finished with value: 37.840404871866895 and parameters: {'lambda_l1': 0.2406432823103

Number of finished trials: 100
Best trial:
  Value: 27.878290494803295
  Params: 
    lambda_l1: 3.24217057813122
    lambda_l2: 9.190559740695685
    n_estimators: 169
    num_leaves: 195
    feature_fraction: 0.4255054464173141
    bagging_fraction: 0.9902779974923297
    bagging_freq: 4
    min_child_samples: 14
